# **LIVE DEMO**

**Importing necessary LIbraries and making certain useful functions for automatic hand detection **

In [8]:
import numpy as np
import cv2 as cv
from keras.models import load_model


# model = load_model("model/incep_resnet_v2.h5")
model = load_model("model/incep_resnet_asl.h5")

def getcnthull(mask_img):
    contours, hierarchy = cv.findContours(mask_img, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    contours = max(contours, key=lambda x: cv.contourArea(x))
    hull = cv.convexHull(contours)
    return contours, hull

OSError: SavedModel file does not exist at: model/incep_resnet_asl.h5/{saved_model.pbtxt|saved_model.pb}

**Created a Dictionary of Alphabets**

Its required here becomes the predicting output is a number ranging from (0 to 29) each represents a character

In [3]:
k = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
label_dict = {}
for i in range(0, 26):
    label_dict[i] = k[i]
label_dict[26] = 'del'
label_dict[27] = 'Space'
label_dict[28] = 'Nothing'
label_dict

{0: 'A',
 1: 'B',
 2: 'C',
 3: 'D',
 4: 'E',
 5: 'F',
 6: 'G',
 7: 'H',
 8: 'I',
 9: 'J',
 10: 'K',
 11: 'L',
 12: 'M',
 13: 'N',
 14: 'O',
 15: 'P',
 16: 'Q',
 17: 'R',
 18: 'S',
 19: 'T',
 20: 'U',
 21: 'V',
 22: 'W',
 23: 'X',
 24: 'Y',
 25: 'Z',
 26: 'del',
 27: 'Space',
 28: 'Nothing'}

**Images were use for testing purpose**

Few downloaded images from google were used, to test the model performance 

In [3]:
img = cv.imread("sign_3.jpg")
hsvim = cv.cvtColor(img, cv.COLOR_BGR2HSV)
lower = np.array([0, 48, 80])
upper = np.array([20, 255, 255])
skinMask = cv.inRange(hsvim, lower, upper)
# skin = cv.bitwise_and(img, img, mask = skinMask)
    
    
    
blurred = cv.blur(skinMask, (2, 2))
# cv.imshow("img", skin)    
    
ret, thresh = cv.threshold(blurred, 0, 255, cv.THRESH_BINARY)

contours, hull = getcnthull(thresh)
cnt = contours

x, y, w, h = cv.boundingRect(cnt)
img = cv.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
cv.imshow("img", img)
# print(img.shape)


hand_img = img[x-100:x+w+100, y-100:y+h+100]
print(hand_img.shape)
hand_img = cv.flip(hand_img, 1)
resized = cv.resize(hand_img, (200, 200))
normalized = resized / 255.0
reshaped = np.reshape(normalized, (1, 200, 200, 3))
        

result = model.predict(reshaped)
idx = np.argmax(result)
# # print(result[0])
print(label_dict[idx])
# # img = cv.putText(img, label_dict[result[0]], (x+10, y+10),cv.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
# cv.putText(img, label_dict[idx], (x+10, y-10),cv.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
# cv.imshow("FInal", img)
        
cv.waitKey()


cv.destroyAllWindows()

error: OpenCV(4.5.1) /tmp/pip-req-build-7m_g9lbm/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


**Final Testing it on Video**

For Live Video testing Run the code part below this

Just Place your hand near the webcamera and make a sign.

**Yay! Thats cool Right**

The idea is that we are first detecting the color of the hand by applying masking technique using opencv
Then drawing a rectangular box around the object detected here the hand itself
Now croppping and resizing the bounding box to (200 X 200) dimension as it need to be of that size accorfing to our model 
The image is then normalized and feed to the DL model and predicted output is noted 

In [7]:
import cv2

IMAGE_SIZE = 200
CROP_SIZE = 400


classes = label_dict
cap = cv2.VideoCapture(0)

while(True):
    # Capture frame-by-frame.
    ret, img = cap.read()
    hsvim = cv.cvtColor(img, cv.COLOR_BGR2HSV)
    lower = np.array([0, 48, 80])
    upper = np.array([20, 255, 255])
    skinMask = cv.inRange(hsvim, lower, upper)
        
    blurred = cv.blur(skinMask, (2, 2))

    ret, thresh = cv.threshold(blurred, 0, 255, cv.THRESH_BINARY)

    contours, hull = getcnthull(thresh)
    cnt = contours

    x, y, w, h = cv.boundingRect(cnt)
    frame = cv.rectangle(img, (x-50, y-20), (x + w + 40, y + h + 10), (0, 255, 0), 2)
    
    """Flipping might be required in some cases"""    
#     frame = cv2.flip(frame, 1)
    """Preprocessing the frame before input to the model"""
    cropped = frame[0:CROP_SIZE, 0:CROP_SIZE]
    resized_frame = cv2.resize(cropped, (200, 200))
    reshaped_frame = (np.array(resized_frame)).reshape((1, 200, 200, 3))
    frame_for_model = reshaped_frame/255

    """Prediction Time"""
    prediction = np.array(model.predict(frame_for_model))
    predicted_class = classes[prediction.argmax()]      # Selecting the max confidence index.
    print(predicted_class)

    cv2.putText(frame, predicted_class , (10, 450), 1, 2, (255, 255, 0), 2, cv2.LINE_AA)
    cv2.imshow('frame', frame)

    # Press q to quit
    k = cv2.waitKey(1) & 0xFF
    if k == ord('q'):
        break

# When everything done, release the capture.
cap.release()
cv2.destroyAllWindows()

B
W
Space
Space
Space
Space
Y
Y
W
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
T
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
J
T
Y
Y
Z
Z
Y
Y
Y
Z
Y
Y
Y
Y
Y
Y
Y
Z
J
Z
Z
Y
T
T
T
T
T
Y
Y
Y
Y
Y
Y
Z
T
J
J
T
V
I
I
I
T
Y
Z
Z
T
Nothing
Nothing
Nothing
Nothing
Nothing
Nothing
Nothing
Nothing
Nothing
Nothing
Nothing
Nothing
Nothing
Nothing
Nothing
J
J
J
J
Y
Y
J
J
Y
Y
J
Y
Y
Z
J
J
J
W
J
J
J
J
J
J
J
J
J
J
Z
J
Z
Z
Z
Y
J
Y
Y
Nothing
Y
Y
Y
Y
Y
J
Z
J
Y
Y
Y
Y
Y
T
Y
Y
Y
Y
Y
W
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
J
J
Nothing
Nothing
P
P
P
J
Nothing
Nothing
Nothing
Nothing
J
Nothing
P
Nothing
Nothing
Nothing
J
Y
J
J
J
J
J
Y
T
Y
Y
Y
Y
Y
Y
Nothing
Nothing
Y
L
Y
Y
Y
Y
J
J
J
J
J
P
J
P
Y
P
P
J
P
P
P
Y
J
J
P
Y
J
J
J
Y
Y
Y
P
Y
J
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
J
W
T
W
Space
Y
Space
Space
Space
Space
Space
Space
Space
Space
Space
Space
Space
Space
Space
Space
Space
Y
Y
W
Z
U
Space
Space
Space
Space
Space
Space
Space
del
Y
Space
Space
Space
Space
Space
Space
Space
Space
Y
Space
Space
Space
Y
Y
Y
Y
W
S
W
Y
J
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
Y
F
N
N
N
T
A
A
Y
Y


In [ ]:
cv.imshow("KK",img)
cv.waitKey(1)